In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb

In [2]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/CZ4041 Machine Learning Project/Colab

Mounted at /content/drive
/content/drive/.shortcut-targets-by-id/1ldppAeVJfaoI4cvwZyTCju9jXbD4LI-H/CZ4041 Machine Learning Project/Colab


In [3]:
data=pd.read_csv('data/train.csv',parse_dates=['date'],index_col=['date'])

The dataset is synthetic and has a solution approximated as 

*predicted Sales = sales(store,item) * salesFactor(month) * salesFactor(day) * salesFactor(year)*                        

SalesFactor is defined as aggregated sales / mean of all sales 


In [7]:


class Model():    

    def preprocessing(self,df):
        data = df.copy()
        data['day'] = data.index.day
        data['month'] = data.index.month
        data['year'] = data.index.year
        data['dayofweek'] = data.index.dayofweek
        data['dayofyear']=data.index.dayofyear
        data['weekofyear']=data.index.weekofyear
        return data
    

    def fit_annual_growthFactor(self,year):
      years = np.arange(self.data.year.min(), year+1)
      annual_growth = self.agg_yearTable.values.squeeze()

      years_diff = year - self.data.year.min() + 1
      exponentialDecay = np.exp((years - year)/years_diff)
      self.yearFactor = np.poly1d(np.polyfit(years[:-1], annual_growth, 2, w=exponentialDecay[:-1]))

    def predict_annual_growthFactor(self,year):
      return self.yearFactor(year)

    def create_base_model(self):
      """
      sales = sales(store,item) * salesFactor(month) * salesFactor(dayofweek) * annualGrowth(year)
      """

      #store item pattern
      self.agg_storeItemTable = pd.pivot_table(self.data, index='store', columns='item', values='sales', aggfunc=np.mean)
      
      # Month Factor
      self.agg_monthTable = pd.pivot_table(self.data, index='month', values='sales', aggfunc=np.mean) / self.base_sales

      # dayofweek Factor
      self.agg_dayTable = pd.pivot_table(self.data, index='dayofweek', values='sales', aggfunc=np.mean) / self.base_sales


      # annualGrowth
      self.agg_yearTable= pd.pivot_table(self.data, index='year', values='sales', aggfunc=np.mean) / self.base_sales
    
        
    def predict(self, item, store, date):

      dow, month, year = date.dayofweek, date.month, date.year

      storeFactor = self.agg_storeItemTable.at[store,item]
      dayFactor = self.agg_dayTable.at[dow, 'sales']
      monthFactor = self.agg_monthTable.at[month, 'sales']


      return storeFactor * monthFactor * dayFactor * self.predict_annual_growthFactor(year)


    def fit(self, data):
        self.data = self.preprocessing(data)
  
            
        self.base_sales = self.data['sales'].mean()

        self.create_base_model()
        self.fit_annual_growthFactor(2018)

    def initTestData(self):
      self.test_data=pd.read_csv('data/test.csv',parse_dates=['date'],index_col=['date'])
      self.test_data=self.preprocessing(self.test_data)
      self.test_data.drop('id',axis=1,inplace=True)
  
    def evaluateTest(self):
        self.initTestData()
        self.predictions=[]

        for index, value in self.test_data.iterrows():
          date, item, store = value[['dayofweek','month','year']], value['item'], value['store']

          sales = self.predict(item, store, date)
          self.predictions.append(sales)


In [8]:
model = Model()

In [9]:
model.fit(data)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: FutureWarning: weekofyear and week have been deprecated, please use DatetimeIndex.isocalendar().week instead, which returns a Series.  To exactly reproduce the behavior of week and weekofyear and return an Index, you may call pd.Int64Index(idx.isocalendar().week)
  if sys.path[0] == '':


In [10]:
model.evaluateTest()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: FutureWarning: weekofyear and week have been deprecated, please use DatetimeIndex.isocalendar().week instead, which returns a Series.  To exactly reproduce the behavior of week and weekofyear and return an Index, you may call pd.Int64Index(idx.isocalendar().week)
  if sys.path[0] == '':


In [ ]:
model.predictions[:10]

[12.45574716474341,
 14.499033535339915,
 14.541905280731559,
 15.550493674251213,
 16.582928885791656,
 17.636835679203784,
 18.683282710587953,
 12.45574716474341,
 14.499033535339915,
 14.541905280731559]